In [1]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import pdist
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import os

In [2]:
DOC_SIZE = 2000  # choose the size of the data here (너무 크면 시간이 오래 걸릴 수도 있음!)
TOP_N = 30

In [5]:
movies_df = pd.read_csv('../data/tmdb_5000_movies.csv')
print(movies.shape)
movies_df.head(3)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [73]:
movies_df[movies_df['overview'].isnull()]
# # we don't want any null values. So replace them with an empty string.
movies_df = movies_df.dropna(subset = ['overview'])
movies_df  = movies_df.reset_index(drop=True)
movies_df.shape

(4800, 21)

In [14]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4800 non-null   int64  
 1   genres                4800 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4800 non-null   int64  
 4   keywords              4800 non-null   object 
 5   original_language     4800 non-null   object 
 6   original_title        4800 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4800 non-null   float64
 9   production_companies  4800 non-null   object 
 10  production_countries  4800 non-null   object 
 11  release_date          4799 non-null   object 
 12  revenue               4800 non-null   int64  
 13  runtime               4800 non-null   float64
 14  spoken_languages      4800 non-null   object 
 15  status               

In [29]:
### TODO 1 #### 
# use CountVectorizer & TfidfVectorizer to construct dtm & dtm_tfidf
count_vect = CountVectorizer()

overview_vect = count_vect.fit_transform(movies_df['overview'])
print(overview_vect.shape)
overview_vect.toarray()
# dtm = movies_df['overview']
# # dtm_tfidf: np = ...  # from csr_matrix to numpy array.  # 이제는 이거 한줄로 끝내기!
# ###############
# print(dtm)  # should be csr sparse matrix
# print(dtm.shape)  # (num_docs, num_terms) 당연히.. 대부분은 0 이겠지! - 다시한번 sparsity를 확인할 수 있다.
# print(dtm_tfidf)  # should be csr sparse matrix
# print(dtm_tfidf.shape)  # (num_docs, num_terms) 당연히.. 대부분은 0 이겠지! - 다시한번 sparsity를 확인할 수 있다.

(4800, 21262)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [21]:
# cosine distance -> this is vectorized...
sims_cosine = cosine_similarity(overview_vect, overview_vect)
# sims_cosine_tfidf = cosine_similarity(dtm_tfidf)  # this may take a while... 
print(sims_cosine.shape)  # (num_doc, num_doc)
# print(sims_cosine_tfidf.shape)  # (num_doc, num_doc)

(4800, 4800)


In [22]:
sims_cosine

array([[1.        , 0.33436693, 0.26462806, ..., 0.32142857, 0.22681618,
        0.19015971],
       [0.33436693, 1.        , 0.35098206, ..., 0.32043497, 0.33783196,
        0.21429865],
       [0.26462806, 0.35098206, 1.        , ..., 0.33078508, 0.22917462,
        0.2191785 ],
       ...,
       [0.32142857, 0.32043497, 0.33078508, ..., 1.        , 0.32991444,
        0.20072414],
       [0.22681618, 0.33783196, 0.22917462, ..., 0.32991444, 1.        ,
        0.26837252],
       [0.19015971, 0.21429865, 0.2191785 , ..., 0.20072414, 0.26837252,
        1.        ]])

In [26]:
# argsort : 인덱스를 가져옴

overview_sim_sorted_ind = sims_cosine.argsort()[:, ::-1]  # argsort
print(overview_sim_sorted_ind[:2])

[[   0 3603 3191 ... 4510 3487 4631]
 [   1  137  274 ... 3608 4510  161]]


In [24]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    print(title_index)
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [34]:
similar_movies = find_sim_movie(movies_df, overview_sim_sorted_ind, 'Superman',10)
similar_movies[['title', 'vote_average']]

[813]
[[ 813   79  311  154  870  599 3223 1995  496 3517]]


,title,vote_average
813,Superman,6.9
79,Iron Man 2,6.6
311,The Adventures of Pluto Nash,4.4
154,Rise of the Guardians,7.1
870,Superman II,6.5
599,Hart's War,5.9
3224,All Is Lost,6.6
1995,Pitch Black,6.7
496,Cloudy with a Chance of Meatballs 2,6.4
3518,The Protector,6.8


- TFIDF

In [35]:
tfidf_vect = TfidfVectorizer()
overview_tfidf_vect = tfidf_vect.fit_transform(movies_df['overview'])

In [36]:
overview_tfidf_vect.shape

(4800, 21262)

In [37]:
sims_cosine_tfidf = cosine_similarity(overview_tfidf_vect, overview_tfidf_vect)

In [38]:
sims_cosine_tfidf

array([[1.        , 0.03666088, 0.01944758, ..., 0.03270512, 0.01738049,
        0.01366019],
       [0.03666088, 1.        , 0.04343445, ..., 0.04862922, 0.03177526,
        0.02104553],
       [0.01944758, 0.04343445, 1.        , ..., 0.0405689 , 0.01699126,
        0.01825306],
       ...,
       [0.03270512, 0.04862922, 0.0405689 , ..., 1.        , 0.04629301,
        0.02385856],
       [0.01738049, 0.03177526, 0.01699126, ..., 0.04629301, 1.        ,
        0.04715488],
       [0.01366019, 0.02104553, 0.01825306, ..., 0.02385856, 0.04715488,
        1.        ]])

In [43]:
overview_sim_sorted_ind_tfidf = sims_cosine_tfidf.argsort()[:, ::-1]  # argsort
print(overview_sim_sorted_ind_tfidf[:2])

[[   0 3603  529 ... 4176 4166 1658]
 [   1 2542 3094 ... 4510 2108 2668]]


In [44]:
similar_movies = find_sim_movie(movies_df, overview_sim_sorted_ind_tfidf, 'Superman',10)
similar_movies[['title', 'vote_average']]

[813]
[[ 813  870   10 2433 2527  914 4656 3541 1368   45]]


,title,vote_average
813,Superman,6.9
870,Superman II,6.5
10,Superman Returns,5.4
2433,Superman IV: The Quest for Peace,4.1
2527,National Lampoon's Vacation,7.1
914,Central Intelligence,6.2
4659,Horse Camp,5.5
3542,R100,5.4
1368,Identity Thief,5.6
45,World War Z,6.7


In [45]:
#초 전체
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()


def weighted_vote_average(record):
    # 개별
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) # Row별 적용

In [46]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)] # 후보군 추출 (2배)
    similar_indexes = similar_indexes.reshape(-1)

    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [48]:
similar_movies = find_sim_movie(movies_df, overview_sim_sorted_ind_tfidf, 'Superman',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
154,Rise of the Guardians,7.1,6.937057
45,World War Z,6.7,6.662025
30,Spider-Man 2,6.7,6.651997
4664,Bronson,6.9,6.628779
2527,National Lampoon's Vacation,7.1,6.621603
2015,Spaceballs,6.7,6.523073
546,Minions,6.4,6.376947
870,Superman II,6.5,6.348972
581,Star Trek: Insurrection,6.3,6.199176
914,Central Intelligence,6.2,6.180342


- 다른 결과 표시 방법

In [78]:
# this is to retrive the titles
indices = pd.Series(data=movies_df.index, index=movies_df['title'])
indices.shape
indices

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4795
Newlyweds                                   4796
Signed, Sealed, Delivered                   4797
Shanghai Calling                            4798
My Date with Drew                           4799
Length: 4800, dtype: int64

In [54]:
indices["Avatar"] 

0

In [79]:
def get_recommendations(title, sims, indices, data) :

    global TOP_N

    # use: np.argsort()[::-1] to recommend titles with scores
    title_idx = indices[title]  # 주어진 영화의 인덱스        
    sims_to_title = sims[title_idx, :] # (4800,)
    
    
    sims_to_title_sorted  = np.sort(sims_to_title)[::-1] # [::-1] 역순
    titles_indices_sorted = np.argsort(sims_to_title)[::-1]
    
    titles_sorted = data['title'][titles_indices_sorted]
        
        
    results = [
        (title, score)
        for title, score in zip(titles_sorted, sims_to_title_sorted)
    ]
    ##############
    return results[:TOP_N]

In [56]:
my_list = [1, 2, 3]
print(my_list[::-1])

[3, 2, 1]


In [62]:
overview_sim_sorted_ind

array([[   0, 3603, 3191, ..., 4510, 3487, 4631],
       [   1,  137,  274, ..., 3608, 4510,  161],
       [   2, 3932,  432, ..., 1111, 1770, 3492],
       ...,
       [4797,   19, 3431, ..., 1038,  801, 3685],
       [4798, 4245, 1199, ..., 4511, 3622, 4510],
       [4799, 2686, 4122, ..., 2614, 3706, 1658]], dtype=int64)

In [63]:
overview_sim_sorted_ind_tfidf

array([[   0, 3603,  529, ..., 4176, 4166, 1658],
       [   1, 2542, 3094, ..., 4510, 2108, 2668],
       [   2, 1343, 4070, ..., 4667, 2032, 4615],
       ...,
       [4797, 4033,  633, ..., 4399, 1658, 2604],
       [4798, 2027, 1796, ..., 4510, 4511, 3706],
       [4799,  868, 4099, ...,  669, 1273, 4074]], dtype=int64)

In [68]:
overview_sim_sorted_ind_tfidf[0, :] 

array([   0, 3603,  529, ..., 4176, 4166, 1658], dtype=int64)

In [83]:
# compare the results of dtm with tfidf
# 이유는 다들 알고 계십니다! ㅎㅎ
print("#### dtm + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Superman", sims_cosine, indices, movies_df)):
  print(idx, title, score)
print("#### tfidf + cosine sim ####")
for idx, (title, score) in enumerate(get_recommendations("Superman", sims_cosine_tfidf, indices, movies_df)):
  print(idx, title, score)

#### dtm + cosine sim ####
0 Superman 1.0000000000000004
1 Iron Man 2 0.4609327677584255
2 The Adventures of Pluto Nash 0.4589285179800713
3 Rise of the Guardians 0.4572380853195265
4 Superman II 0.4556611884328836
5 Hart's War 0.4448508420389507
6 All Is Lost 0.4448396935314748
7 Pitch Black 0.4421388081402907
8 Cloudy with a Chance of Meatballs 2 0.4400152785735414
9 The Protector 0.4391485006492564
10 Semi-Pro 0.4381780460041329
11 The Four Feathers 0.4370956778314645
12 Invaders from Mars 0.4349679071977275
13 Gladiator 0.43404648300994053
14 The Midnight Meat Train 0.4323586743502425
15 The Shadow 0.431974468860803
16 54 0.42875376060629783
17 Once in a Lifetime: The Extraordinary Story of the New York Cosmos 0.4232029952561754
18 The Tree of Life 0.4230143927463794
19 Dinner Rush 0.4224711211252415
20 Air Force One 0.4221585268381751
21 The Visitors II: The Corridors of Time 0.42176248989638454
22 Me You and Five Bucks 0.4208834246473209
23 F.I.S.T. 0.42077622506380674
24 The Lon